### Step 1: Download data from open-bus GTFS archive

GET https://s3-eu-west-1.amazonaws.com/s3.obus.hasadna.org.il/2019-03-29.zip

In [54]:
import datetime
import urllib.request
import tempfile
import os
import ot_gtfs_utils

In [56]:
df = ot_gtfs_utils.get_or_create_daily_trips()

Return from pickle /tmp/2019-03-29/2019-03-29.pickle


In [58]:
df.shape

(3326, 36)

In [4]:
feed = gtfs_utils.get_partridge_feed_by_date(daily_gtfs, datetime.date.today())

The feed has in it all the (standard) files in the original GTFS zip, as [pandas](https://github.com/pandas-dev/pandas) DataFrames.

In [5]:
[x for x in dir(feed) if not x.startswith('_')]

['agency',
 'calendar',
 'calendar_dates',
 'fare_attributes',
 'fare_rules',
 'feed_info',
 'frequencies',
 'get',
 'routes',
 'shapes',
 'stop_times',
 'stops',
 'transfers',
 'trips']

In [6]:
trips_and_routes = feed.trips.merge(feed.routes, on="route_id")

In [7]:
train_trips_and_routes = trips_and_routes[trips_and_routes['agency_id']=='2']

In [8]:
train_trips_and_routes.shape

(408, 12)

In [9]:
train_stops_trip_routes = feed.stop_times.merge(train_trips_and_routes, on="trip_id")

In [10]:
train_stops_trip_routes.shape

(3326, 19)

In [11]:
train_stops_trip_routes.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled,route_id,service_id,trip_headsign,direction_id,shape_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_color
0,30124744_290319,13980.0,13980.0,37358,1,0,1,0,21388,36091,NaN,1,81882,2,21,תל אביב מרכז-תל אביב יפו<->נהריה-נהריה,46021-2-42,2,NaN
1,30124744_290319,14400.0,14400.0,37362,2,0,0,9200,21388,36091,NaN,1,81882,2,21,תל אביב מרכז-תל אביב יפו<->נהריה-נהריה,46021-2-42,2,NaN
2,30124744_290319,15120.0,15120.0,37366,3,0,0,27400,21388,36091,NaN,1,81882,2,21,תל אביב מרכז-תל אביב יפו<->נהריה-נהריה,46021-2-42,2,NaN
3,30124744_290319,15720.0,15720.0,37368,4,0,0,41100,21388,36091,NaN,1,81882,2,21,תל אביב מרכז-תל אביב יפו<->נהריה-נהריה,46021-2-42,2,NaN
4,30124744_290319,16320.0,16320.0,37372,5,0,0,52100,21388,36091,NaN,1,81882,2,21,תל אביב מרכז-תל אביב יפו<->נהריה-נהריה,46021-2-42,2,NaN


In [12]:
latest_train_data = train_stops_trip_routes.merge(feed.calendar, on="service_id")

In [13]:
latest_train_data.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled,route_id,service_id,...,route_color,sunday,monday,tuesday,wednesday,thursday,friday,saturday,start_date,end_date
0,30124744_290319,13980.0,13980.0,37358,1,0,1,0,21388,36091,...,NaN,0,0,0,0,0,1,0,2019-03-29,2019-03-29
1,30124744_290319,14400.0,14400.0,37362,2,0,0,9200,21388,36091,...,NaN,0,0,0,0,0,1,0,2019-03-29,2019-03-29
2,30124744_290319,15120.0,15120.0,37366,3,0,0,27400,21388,36091,...,NaN,0,0,0,0,0,1,0,2019-03-29,2019-03-29
3,30124744_290319,15720.0,15720.0,37368,4,0,0,41100,21388,36091,...,NaN,0,0,0,0,0,1,0,2019-03-29,2019-03-29
4,30124744_290319,16320.0,16320.0,37372,5,0,0,52100,21388,36091,...,NaN,0,0,0,0,0,1,0,2019-03-29,2019-03-29


In [14]:
stop_ids = latest_train_data['stop_id'].unique()

In [15]:
train_stops = feed.stops[feed.stops.stop_id.isin(stop_ids)]

In [16]:
latest_train_data_with_stops = latest_train_data.merge(train_stops, on="stop_id")

In [17]:
latest_train_data_with_stops.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled,route_id,service_id,...,start_date,end_date,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,zone_id
0,30124744_290319,13980.0,13980.0,37358,1,0,1,0,21388,36091,...,2019-03-29,2019-03-29,17038,תל אביב מרכז,רחוב:מסילת ברזל עיר: תל אביב יפו רציף: קומה...,32.083715,34.798247,0,NaN,5000
1,30124854_290319,3180.0,3180.0,37358,1,0,1,0,21386,36086,...,2019-03-29,2019-03-29,17038,תל אביב מרכז,רחוב:מסילת ברזל עיר: תל אביב יפו רציף: קומה...,32.083715,34.798247,0,NaN,5000
2,30124855_290319,6780.0,6780.0,37358,1,0,1,0,21386,36086,...,2019-03-29,2019-03-29,17038,תל אביב מרכז,רחוב:מסילת ברזל עיר: תל אביב יפו רציף: קומה...,32.083715,34.798247,0,NaN,5000
3,30124856_290319,10380.0,10380.0,37358,1,0,1,0,21386,36086,...,2019-03-29,2019-03-29,17038,תל אביב מרכז,רחוב:מסילת ברזל עיר: תל אביב יפו רציף: קומה...,32.083715,34.798247,0,NaN,5000
4,31552312_290319,19800.0,19800.0,37358,11,1,0,122600,21390,36047,...,2019-03-29,2019-03-29,17038,תל אביב מרכז,רחוב:מסילת ברזל עיר: תל אביב יפו רציף: קומה...,32.083715,34.798247,0,NaN,5000


In [18]:
from IPython.display import display
import pandas as pd

In [19]:
pd.options.display.max_columns = None

In [20]:
latest_train_data_with_stops.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled,route_id,service_id,trip_headsign,direction_id,shape_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_color,sunday,monday,tuesday,wednesday,thursday,friday,saturday,start_date,end_date,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,zone_id
0,30124744_290319,13980.0,13980.0,37358,1,0,1,0,21388,36091,NaN,1,81882,2,21,תל אביב מרכז-תל אביב יפו<->נהריה-נהריה,46021-2-42,2,NaN,0,0,0,0,0,1,0,2019-03-29,2019-03-29,17038,תל אביב מרכז,רחוב:מסילת ברזל עיר: תל אביב יפו רציף: קומה...,32.083715,34.798247,0,NaN,5000
1,30124854_290319,3180.0,3180.0,37358,1,0,1,0,21386,36086,NaN,1,81882,2,21,תל אביב מרכז-תל אביב יפו<->נהריה-נהריה,46021-2-41,2,NaN,0,0,0,0,0,1,0,2019-03-29,2019-03-29,17038,תל אביב מרכז,רחוב:מסילת ברזל עיר: תל אביב יפו רציף: קומה...,32.083715,34.798247,0,NaN,5000
2,30124855_290319,6780.0,6780.0,37358,1,0,1,0,21386,36086,NaN,1,81882,2,21,תל אביב מרכז-תל אביב יפו<->נהריה-נהריה,46021-2-41,2,NaN,0,0,0,0,0,1,0,2019-03-29,2019-03-29,17038,תל אביב מרכז,רחוב:מסילת ברזל עיר: תל אביב יפו רציף: קומה...,32.083715,34.798247,0,NaN,5000
3,30124856_290319,10380.0,10380.0,37358,1,0,1,0,21386,36086,NaN,1,81882,2,21,תל אביב מרכז-תל אביב יפו<->נהריה-נהריה,46021-2-41,2,NaN,0,0,0,0,0,1,0,2019-03-29,2019-03-29,17038,תל אביב מרכז,רחוב:מסילת ברזל עיר: תל אביב יפו רציף: קומה...,32.083715,34.798247,0,NaN,5000
4,31552312_290319,19800.0,19800.0,37358,11,1,0,122600,21390,36047,NaN,0,81881,2,21,נהריה-נהריה<->תל אביב מרכז-תל אביב יפו,46021-1-55,2,NaN,0,0,0,0,0,1,0,2019-03-29,2019-03-29,17038,תל אביב מרכז,רחוב:מסילת ברזל עיר: תל אביב יפו רציף: קומה...,32.083715,34.798247,0,NaN,5000


In [21]:
latest_train_data_with_stops.to_pickle('daily_gtfs.pickle')

In [ ]:
latest_train_data_with_stops[]

In [42]:
train_stops.stop_id.unique()

array(['37290', '37292', '37294', '37296', '37298', '37300', '37302',
       '37304', '37306', '37308', '37310', '37312', '37314', '37316',
       '37318', '37320', '37322', '37324', '37326', '37328', '37330',
       '37332', '37334', '37336', '37338', '37340', '37342', '37344',
       '37346', '37348', '37352', '37354', '37356', '37358', '37360',
       '37362', '37364', '37366', '37368', '37370', '37372', '37374',
       '37376', '37378', '37380', '37382', '37384', '37386', '37388',
       '37392', '37394', '37396', '38643', '39260', '40581', '40582',
       '40583', '40584', '40640', '41291', '41292', '41293', '42416',
       '42417', '42419', '42507'], dtype=object)

In [46]:
train_stops

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,zone_id
23230,37290,17106,שדרות,רחוב:מסילת ברזל עיר: שער הנגב רציף: קומה: ...,31.515877,34.586059,0,NaN,1926
23231,37292,17104,תל אביב ההגנה,רחוב:מסילת ברזל עיר: תל אביב יפו רציף: קומה...,32.053978,34.784839,0,NaN,5000
23232,37294,17102,רשל''צ משה דיין,רחוב:מסילת ברזל עיר: ראשון לציון רציף: קומה...,31.987987,34.757392,0,NaN,8300
23233,37296,17100,הוד השרון,רחוב:מסילת ברזל עיר: הוד השרון רציף: קומה: ...,32.170213,34.901609,0,NaN,9700
23234,37298,17098,ראשונים,רחוב:מסילת ברזל עיר: ראשון לציון רציף: קומה...,31.948927,34.802622,0,NaN,8300
23235,37300,17096,יבנה מערב,רחוב:מסילת ברזל עיר: יבנה רציף: קומה: תחנ...,31.891676,34.731526,0,NaN,14067
23236,37302,17094,ראש העין-צפון,רחוב:מסילת ברזל עיר: ראש העין רציף: קומה: ...,32.120865,34.934527,0,NaN,2640
23237,37304,17092,כפר סבא,רחוב:מסילת ברזל עיר: הוד השרון רציף: קומה: ...,32.167369,34.917428,0,NaN,9700
23238,37306,17090,נת-בג,רחוב:מסילת ברזל עיר: נמל תעופה בן גוריון רציף...,32.000360,34.870481,0,NaN,1748
23239,37308,17088,להבים רהט,רחוב:מסילת ברזל עיר: להבים רציף: קומה: תח...,31.369907,34.798040,0,NaN,1271


Stops do not contain a GTFS id  they can identified using lat long data

In [63]:
train_stops[['stop_name', 'stop_code']].sort_values('stop_name')

,stop_name,stop_code
24366,אופקים,17109
25867,אחיהוד,17116
23248,אשדוד עד הלום,17070
23247,אשקלון,17072
23250,באר יעקב,17066
23241,באר שבע מרכז,17084
23242,באר שבע-צפון,17082
23266,בית יהושע,17032
25373,בית שאן,17111
23246,בית שמש,17074


From Rosh haayin to Tel Aviv Univ. today

In [69]:
 d = latest_train_data_with_stops

In [78]:
stops_departure = d[d.stop_code == '17094']

In [77]:
stops_arrival = d[d.stop_code == '17036']

In [91]:
merge_stops = stops_departure.merge(stops_arrival, on="trip_id")

In [102]:
daily_stops_from_to = merge_stops[merge_stops.stop_sequence_y > merge_stops.stop_sequence_x]

All trips with Rosh Haayin as departure and TA Univ. as arrival done today

In [54]:
pd.options.display.max_rows = 100

In [106]:
now = datetime.datetime.now()

In [111]:
now_secs_from_midnight = 3600 * now.hour + 60 * now.minute + now.second

In [112]:
now_secs_from_midnight

58639

In [ ]:
delta_stops_from_to = daily_stops_from_to.arrival_time_x > now_secs_from_midnight - 3000 